In [213]:
using PlotlyJS
using Infiltrator
# using XLSX
include("../model/utils.jl")
include("../model/unit_commitment.jl")
include("../model/economic_dispatch.jl")
include("./plotting.jl")
include("./processing.jl")
# XLSX.writetable(joinpath(folder_path,"LOL_CUR.xlsx"), "LOL_CUR" => change_type(LOL_CUR, Symbol, string))

calculate_reserve_gcd_KPI (generic function with 1 method)

In [246]:
folder_path = joinpath("..","output", "solutions_v21.0u")
aux = "all"
# aux = join(string.("n_",[7, 228, 259, 289]),"_")
day_ = 7
gcdi_KPI_adequacy = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcdi_KPI_adequacy.parquet"))
gcd_KPI_adequacy = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcd_KPI_adequacy.parquet"))
KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_KPI_reserve.parquet"))
gcdi_KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcdi_KPI_reserve.parquet"))
gcd_KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcd_KPI_reserve.parquet"))


;

../output/solutions_v21.0u/all_gcdi_KPI_adequacy.parquet
../output/solutions_v21.0u/all_gcd_KPI_adequacy.parquet
../output/solutions_v21.0u/all_KPI_reserve.parquet
../output/solutions_v21.0u/all_gcdi_KPI_reserve.parquet
../output/solutions_v21.0u/all_gcd_KPI_reserve.parquet


In [247]:
reserve_thrs = 0.1+1e-10
group_by = [:configuration, :day, :iteration]

ov = leftjoin(
    gcdi_KPI_adequacy[:,union(group_by,[:objective_value, :OPEX, :ENS_MWh])],
    combine(groupby(KPI_reserve, group_by), :required_r_relative => (x-> all(abs.(x) .<= reserve_thrs))  => :required_reserve_within_margins),
    on = group_by
)

eov = combine(groupby(ov, [:configuration, :day, :required_reserve_within_margins]), [:objective_value, :OPEX, :ENS_MWh] .=> mean .=> [:EOV, :EOPEX, :EENS])
sort!(transform!(eov, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu), :mu)
;

In [248]:
plot(eov, x = :mu, y = :EOV, color = :day, facet_col = :required_reserve_within_margins)
# p2 = plot(eov, x = :mu, y = :EENS, color = :day, facet_col = :required_reserve_within_margins)


data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

In [249]:
plot(eov, x = :mu, y = :EENS, color = :day, facet_col = :required_reserve_within_margins)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

In [250]:
function redispatch_prob(r_relative_min, r_relative_max, required_reserve, number_key)
    out =  combine(groupby(required_reserve, [:configuration, :mu]), [:required_r_relative, :redispatch_needed] => ((x,y) -> 100*sum((x.>=r_relative_min).*(x.<=r_relative_max).*y)/sum((x.>=r_relative_min).*(x.<=r_relative_max))) => :redispatch_probability)
    out.r_max_relative .= number_key
    return out
end

redispatch_prob (generic function with 1 method)

KPI_reserve

In [251]:
KPI_reserve

Row,configuration,day,iteration,hour,demand_MW,LOL_MW,LGEN_MW,demand_uc_MW,required_r_MW,required_r_relative,redispatch_MW,redispatch_relative,redispatch_needed,required_r_up_MW,required_r_dn_MW,production_MW,charge_MW,discharge_MW,delivered_r_up_MW,delivered_r_dn_MW,delivered_r_up_ratio,delivered_r_dn_ratio,mu
,Symbol,Int64,Symbol,Int64?,Float64?,Float64?,Float64?,Int64?,Float64?,Float64?,Float64?,Float64?,Bool?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64
1,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,169,1496.1,0.0,25.898,1522,-25.898,-0.0170158,-25.898,-0.0170158,true,-0.0,25.898,1438.37,0.0,83.6266,0.0,-0.0,NaN,-0.0,0.0
2,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,170,1481.34,0.0,0.0,1451,30.3445,0.0209128,0.0,0.0,false,30.3445,-0.0,1421.77,0.0,59.5762,30.3445,-0.0,1.0,NaN,0.0
3,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,171,1509.47,0.0,0.0,1461,48.4743,0.0331789,0.0,0.0,false,48.4743,-0.0,1396.81,0.0,112.667,48.4743,-0.0,1.0,NaN,0.0
4,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,172,1438.66,0.0,0.0,1531,-92.3403,-0.0603137,0.0,0.0,false,-0.0,92.3403,1425.4,0.0,13.2618,-0.0,92.3403,NaN,1.0,0.0
5,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,173,1673.51,0.0,0.0,1690,-16.4925,-0.00975887,0.0,0.0,false,-0.0,16.4925,1425.16,0.0,248.352,-0.0,16.4925,NaN,1.0,0.0
6,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,174,1904.24,0.0,0.0,1911,-6.76186,-0.00353839,0.0,0.0,false,-0.0,6.76186,1477.01,0.0,427.228,-0.0,6.76186,NaN,1.0,0.0
7,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,175,2041.76,0.0,0.0,2150,-108.239,-0.0503436,0.0,0.0,false,-0.0,108.239,1485.19,0.0,556.57,-0.0,108.239,NaN,1.0,0.0
8,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,176,2174.1,0.0,0.0,2248,-73.8954,-0.0328716,0.0,0.0,false,-0.0,73.8954,1651.98,0.0,522.129,-0.0,73.8954,NaN,1.0,0.0
9,base_ramp_storage_envelopes_up_0_dn_0,7,demand_86,177,2195.18,0.0,309.191,2337,-141.816,-0.0606828,-309.191,-0.132303,true,-0.0,141.816,2705.54,201.164,0.0,167.376,-0.0,-Inf,-0.0,0.0


In [252]:
combine(groupby(KPI_reserve, [:configuration, :mu]), [:required_r_relative, :redispatch_needed] => ((x,y) -> sum((x.>=0).*(x.<=0.1))) => :redispatch_probability)
# 
# CSV.write("KPI_reserve_v12_0.csv", KPI_reserve)

Row,configuration,mu,redispatch_probability
,Symbol,Float64,Int64
1,base_ramp_storage_envelopes_up_0_dn_0,0.0,1180
2,base_ramp_storage_envelopes_up_0_2_dn_0_2,0.2,1180
3,base_ramp_storage_envelopes_up_0_4_dn_0_4,0.4,1180
4,base_ramp_storage_envelopes_up_0_6_dn_0_6,0.6,1180
5,base_ramp_storage_envelopes_up_0_8_dn_0_8,0.8,1180
6,base_ramp_storage_envelopes_up_1_dn_1,1.0,1180


In [253]:
redispatch_probability = vcat(
    vcat([redispatch_prob(0, r_max, KPI_reserve, r_max) for r_max in 0.0:0.01:0.15]...),
    vcat([redispatch_prob(-r_max, 0, KPI_reserve, -r_max) for r_max in 0.0:0.01:0.15]...)
)
sort!(redispatch_probability, :r_max_relative)
# redispatch_probability = vcat([redispatch_prob(-r_max, r_max, required_reserve, r_max) for r_max in 0.0:0.01:0.2]...)
p1 = plot(redispatch_probability, x = :r_max_relative, y = :redispatch_probability, color = :mu, Layout(title = "Redispatch probability", yaxis_title = "probability  [%]", xaxis_title = "required reserve [%]"), mode="markers+lines")
p2 = plot(filter(:mu => x -> ismissing(x) ? false : x == 1, redispatch_probability), x = :r_max_relative, y = :redispatch_probability, Layout(title = "Redispatch probability mu = 1 ", yaxis_title = "probability  [%]", xaxis_title = "required reserve [%]"), name = "1.0", mode="markers+lines", marker=attr(color="black"))
[p1 p2]

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields marker, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [254]:
p2 = plot(filter(:day => x -> x in [day_], KPI_reserve), x = :required_r_relative, y = :redispatch_MW, color = :mu, mode = "markers", Layout(height=800))

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields height, legend, margin, template, xaxis, and yaxis"

In [255]:
p2 = plot(filter(:day => x -> x in [day_], KPI_reserve), x = :required_r_relative, y = :mu, facet_col = :redispatch_needed, facet_row = :day, mode = "markers", Layout(height=800))
# p2 = plot(filter(:mu => x -> ismissing(x) ? false : x == 1, KPI_reserve), x = :required_r_relative, y = :mu, facet_col = :redispatch_needed, mode = "markers", Layout(height=800))

data: [
  "scatter with fields mode, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, legend, margin, template, xaxis, xaxis2, yaxis, and yaxis2"

In [256]:
# KPI_reserve[:KPI_reserve.mu .== 1]

In [257]:
p1 = plot(gcd_KPI_adequacy, x = :mu, y = :EOV, color = :day)
p2 = plot(gcd_KPI_adequacy, x = :mu, y = :EENS, color = :day)
p3 = plot(gcd_KPI_adequacy, x = :mu, y = :ECUR, color = :day)
p4 = plot(gcd_KPI_adequacy, x = :mu, y = :ELGEN, color = :day)
# p4 = plot(gcd_KPI_adequacy, x = :mu, y = :ELGEN, color = :day)
# p3 = plot(KPI, x = :mu, y = :LOLE, color = :day)
[p1 p2 p4]
# savefig([p1 p2 p3 p4], "adecuacy KPIs v1.png", width=12*scale, height=6*scale)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, xaxis3, yaxis1, yaxis2, and yaxis3"

In [258]:
p1 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_up_ratio, color = :day)
p2 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_dn_ratio, color = :day)
p3 = plot(gcd_KPI_reserve, x = :mu, y = :E_delivered_r_up_MWh, color = :day)
p4 = plot(gcd_KPI_reserve, x = :mu, y = :E_deliverered_dn_MWh, color = :day)
# [p1 p2; p3 p4]
[p1 p2]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [259]:
p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :mu, color = :iteration)
p2 = plot(gcdi_KPI_adequacy, y = :ENS_MWh, x = :mu, color = :iteration)
p3 = plot(gcdi_KPI_adequacy, y = :LGEN_MWh, x = :mu, color = :iteration)
p4 = plot(gcdi_KPI_adequacy, y = :CUR_MWh, x = :mu, color = :iteration)
[p1 p2; p3 p4]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and y

In [260]:
p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :mu, facet_col = :day, color = :iteration)
[p1;]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and y

In [261]:
# function check_loq_value(df, field, value, accuracy)
#     filter = (df[!,field] .> value) .& (df[!,field] .< Inf) .& (.!isnan.(df[!,field]))
#     filter = (df[!,field].*filter .-value) .< accuracy
#     println(all(filter))
#     return df[.!filter,:]

# end
# # Check
# accuracy = 0.000001
# check_loq_value(KPI_reserve, :delivered_r_up_ratio, 1, accuracy)
# check_loq_value(KPI_reserve, :delivered_r_dn_ratio, 1, accuracy)
;
# balance = (KPI_reserve.required_r_up_MW .-KPI_reserve.required_r_dn_MW) .- (KPI_reserve.delivered_r_up_MW .- KPI_reserve.delivered_r_dn_MW .+ KPI_reserve.LOL_MW.*(required.>=0))
# println(all(abs.(balance) .<= accuracy))
# Todo: re-calcualte balance balance
# p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration) 
# p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_valuef, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration)
# p1 = plot(gcdi_KPI_adequacy, y = :objective_value, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# p2 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# p3= plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# relayout([p1;p2;p3], height=800)